In [7]:
# ORIGINAL CODE : https://machinelearningmastery.com/weighted-average-ensemble-with-python/
# By Jason Brownlee - MAY 2021

# Modifed by MAHAYASA ADIPUTRA - OCT 2022
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix ,accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as mt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import VotingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
le = LabelEncoder()
start_time = time.time()
# get the dataset
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/mobile-churn.csv')
#filling missing value with mean
dataset = dataset.fillna(dataset.mean())
X=dataset.drop(['churn','year','month','user_account_id'],axis=1)
y=dataset["churn"]
#y = le.fit_transform(y)
#imblearn SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)
#smote_enn = SMOTEENN()
#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# get a list of base models
def get_models():
	models = list()
	models.append(('xb', XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)))
	models.append(('rf', RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)))
#	models.append(('ab', AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')))
	return models
 
# evaluate each base model
def evaluate_models(models, X,y):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# evaluate the model
		acc=cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
		# store the performance
		scores.append(mean(acc))
		# report model performance
	return scores



# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X,y)
print(scores)
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
score=cross_val_score(ensemble, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

[0.961010867461701, 0.9619312135553324]
F1 score: 0.917
STD F1 Score: 0.001


In [8]:
rc=cross_val_score(ensemble, X, y, scoring='recall', cv=cv, n_jobs=-1)
rc=mean(rc)
rc

0.9303318581925722

In [9]:
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(ensemble, X, y, cv=5, scoring=scorer)
sp=mean(sp)
sp

0.8380912770750891

In [10]:
auc=cross_val_score(ensemble, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
auc=mean(auc)
auc

0.965000423690985

In [11]:
import math

gmean=rc*sp
sqrtg = math.sqrt(gmean)
sqrtg

0.8830079360550808

In [12]:
X

,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,...,last_100_reloads_sum,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage
0,1000,0,1,0.050000,0.000000,1,1,0,0,66,...,12.010000,75.270000,0.000000,63.430000,0.000000,210,1,84,0,0.0
1,1000,0,25,28.310000,3.450000,1,0,0,0,1276,...,0.000000,13.380000,11.180000,2.000000,11.180000,0,0,0,0,0.0
2,1005,0,8,15.620000,1.970000,1,0,0,0,1276,...,0.000000,30.000000,0.000000,0.000000,10.450000,0,0,0,0,0.0
3,1013,0,11,5.620000,0.000000,1,0,0,0,1276,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0.0
4,1032,0,2,5.860000,0.150000,1,0,0,1,17,...,6.000000,2.580000,0.000000,1.000000,0.000000,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101420,15947,0,8,0.000000,0.000000,0,0,0,1,8,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0.0
101421,82,0,1,0.287880,25.827049,1,1,0,1,3,...,96.095034,426.824501,0.000000,335.776822,0.000000,533,1,490,0,0.0
101422,1295,0,83,7.542798,0.000000,0,0,0,0,1276,...,0.000000,4.169892,0.000000,2.363224,0.000000,0,0,0,0,0.0
101423,890,0,2,10.770470,1.320111,1,1,0,1,7,...,12.706442,13.976436,2.620311,0.379124,2.620311,4,0,1,0,0.0
